In [1]:
import os
import numpy as np

try:
    import pandas as pd
except ImportError:
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pandas"])
finally:
    import pandas as pd

try:
    import pyarrow
except ImportError:
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pyarrow"])
finally:
    import pyarrow

try:
    from tqdm import tqdm
except ImportError:
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "tqdm"])
finally:
    from tqdm import tqdm

try:
    from scipy.spatial.transform import Rotation
except ImportError:
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "scipy"])
finally:
    from scipy.spatial.transform import Rotation

try:
    import antropy
except ImportError:
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "antropy"])
finally:
    import antropy 

|Index| 한글 | English |
|:--| :--- | :--- |
|0| 시간 | Time |
|1| 경과 | Time_sec |
|2| 좌안-PD | Left Pupil Diameter |
|3| 우안-PD | Right Pupil Diameter |
|4| 좌안-열림 | Left Eye Openness |
|5| 우안-열림 | Right Eye Openness |
|6| 좌안-동공위치-X | left_eye_gaze_x |
|7| 좌안-동공위치-Y | left_eye_gaze_y |
|8| 좌안-동공위치-Z | left_eye_gaze_z-x |
|9| 우안-동공위치-X | left_eye_gaze_x |
|10| 우안-동공위치-Y | left_eye_gaze_y |
|11| 우안-동공위치-Z | left_eye_gaze_z-x |
|12| 좌안-시선방향(전역)-X | left_eye_direction_x |
|13| 좌안-시선방향(전역)-Y | left_eye_direction_y |
|14| 좌안-시선방향(전역)-Z | left_eye_direction_z |
|15| 우안-시선방향(전역)-X | right_eye_direction_x |
|16| 우안-시선방향(전역)-Y | right_eye_direction_y |
|17| 우안-시선방향(전역)-Z | right_eye_direction_z |
|18| 종합-시선방향(전역)-X | combined_eye_direction_x |
|19| 종합-시선방향(전역)-Y | combined_eye_direction_y |
|20| 종합-시선방향(전역)-Z | combined_eye_direction_z |
|21| 좌안-시선방향(지역)-X | left_eye_direction_cam_x |
|22| 좌안-시선방향(지역)-Y | left_eye_direction_cam_y |
|23| 좌안-시선방향(지역)-Z | left_eye_direction_cam_z |
|24| 우안-시선방향(지역)-X | right_eye_direction_cam_x |
|25| 우안-시선방향(지역)-Y | right_eye_direction_cam_y |
|26| 우안-시선방향(지역)-Z | right_eye_direction_cam_z |
|27| 종합-시선방향(지역)-X | combined_eye_direction_cam_x |
|28| 종합-시선방향(지역)-Y | combined_eye_direction_cam_y |
|29| 종합-시선방향(지역)-Z | combined_eye_direction_cam_z |
|30| 머리-위치-X | cam_pos_x |
|31| 머리-위치-Y | cam_pos_y |
|32| 머리-위치-Y | cam_pos_z |
|33| 머리-회전-X | cam_rot_x |
|34| 머리-회전-Y | cam_rot_y |
|35| 머리-회전-Z | cam_rot_z |
|36| 멀미 여부 | VR sickness response |
|37| FMS | Score |
|38| RR 간격 | RR_interval |
|39| 심박수 | heart_rate |
|40| HRV | HRV |
|41| Optical flow | Optical_Flow |

In [2]:
from IPython.display import display

Lvs = ['Lv1', 'Lv2']

root_360_raw = "/Volumes/LocalHDD/2024-NRF-360Videos-H.T.Kim/raw/"
root_360_features = "/home/shared_home/2026-NRF-360Videos/data/"

subroot_list = sorted(os.listdir(root_360_raw))
subroot_list = [item for item in subroot_list if not item.startswith('.')]
display(subroot_list)

['P01_0704',
 'P02_0704',
 'P03_0704',
 'P04_0704',
 'P05_0704',
 'P06_0704',
 'P07_0704',
 'P08_0704',
 'P09_0704',
 'P10_0704']

In [18]:
results = []

def convert_euler_to_quaternion(euler_x, euler_y, euler_z):
    rad_x = np.deg2rad(euler_x)
    rad_y = np.deg2rad(euler_y)
    rad_z = np.deg2rad(euler_z)

    cz = np.cos(rad_z * 0.5)
    sz = np.sin(rad_z * 0.5)
    cx = np.cos(rad_x * 0.5)
    sx = np.sin(rad_x * 0.5)
    cy = np.cos(rad_y * 0.5)
    sy = np.sin(rad_y * 0.5)

    qw = cz * cx * cy + sz * sx * sy
    qx = cz * sx * cy - sz * cx * sy
    qy = cz * cx * sy + sz * sx * cy
    qz = sz * cx * cy - cz * sx * sy
    
    return qx, qy, qz, qw

full_dataset = []

for subroot in tqdm(subroot_list, total=10):
    subject_id = subroot.split('_')[0]
    lv_list = sorted(os.listdir(os.path.join(root_360_raw, subroot)))
    for l_id, lv in enumerate(lv_list):
        raw_data = pd.read_csv(os.path.join(root_360_raw, subroot, lv, 'raw.csv'))
        raw_data.columns = [c.lower() for c in raw_data.columns]
        raw_data['ID'] = subject_id
        raw_data["Lv"] = lv
        raw_data['frame'] = range(len(raw_data))
        raw_data['Elapsed'] = raw_data["time_sec"]
        
        gaze_vec_local = raw_data[['combined_eye_direction_cam_x',\
                                   'combined_eye_direction_cam_y',\
                                   'combined_eye_direction_cam_z']].values
        # 3DOF -> 2DOF
        raw_data["gaze-azimuth"] = np.rad2deg(np.arctan2(gaze_vec_local[:, 0], gaze_vec_local[:, 2]))
        raw_data["gaze-elevation"] = np.rad2deg(np.arcsin(np.clip(gaze_vec_local[:,1], -1, 1)))

        condition = (raw_data['gaze-azimuth'] == 0) & (raw_data['gaze-elevation'] == 0)
        raw_data.loc[condition, ['gaze-azimuth', 'gaze-elevation']] = np.nan
        
        # 선형 속도
        qx, qy, qz, qw = convert_euler_to_quaternion(
            raw_data["cam_rot_x"],
            raw_data["cam_rot_y"],
            raw_data["cam_rot_z"]
        )
        raw_data = raw_data.assign(CVT_H_qut_x=qx, CVT_H_qut_y=qy, CVT_H_qut_z=qz, CVT_H_qut_w=qw)

        raw_data["prev_Elapsed"] = raw_data["Elapsed"].shift(1)
        raw_data["prev_H_pos_x"] = raw_data["cam_pos_x"].shift(1)
        raw_data["prev_H_pos_y"] = raw_data["cam_pos_y"].shift(1)
        raw_data["prev_H_pos_z"] = raw_data["cam_pos_z"].shift(1)
        raw_data["prev_H_qut_x"] = raw_data["CVT_H_qut_x"].shift(1)
        raw_data["prev_H_qut_y"] = raw_data["CVT_H_qut_y"].shift(1)
        raw_data["prev_H_qut_z"] = raw_data["CVT_H_qut_z"].shift(1)
        raw_data["prev_H_qut_w"] = raw_data["CVT_H_qut_w"].shift(1)
        raw_data.loc[0, "prev_Elapsed"] = 0

        delta_H_pos = raw_data[["cam_pos_x", "cam_pos_y", "cam_pos_z"]].values - \
                    raw_data[["prev_H_pos_x", "prev_H_pos_y", "prev_H_pos_z"]]
        
        prev_H_quaternions = raw_data[["prev_H_qut_x", "prev_H_qut_y", "prev_H_qut_z", "prev_H_qut_w"]].values
        prev_H_quaternions[0] = [0, 0, 0, 1]
        prev_H_rotations = Rotation.from_quat(prev_H_quaternions)
        curr_H_quaternions = raw_data[["CVT_H_qut_x", "CVT_H_qut_y", "CVT_H_qut_z", "CVT_H_qut_w"]].values
        curr_H_rotations = Rotation.from_quat(curr_H_quaternions)
        delta_H_rotations = prev_H_rotations.inv() * curr_H_rotations

        raw_data['dt'] = raw_data['time_sec'].diff()
        raw_data['dt'] = raw_data['dt'].replace(0, 0.0001)
        
        l2norm_H_pos = np.linalg.norm(delta_H_pos, axis=1)
        l2norm_H_rot = np.linalg.norm(delta_H_rotations.as_rotvec(), axis=1)

        raw_data['head-speed'] = l2norm_H_pos / raw_data['dt']
        raw_data['head-ang_vel'] = l2norm_H_rot / raw_data['dt']


        # raw_data['Elapsed']가 30초 간격에 가까울 때마다 (30.01 이렇게 측정될 수 있으니),
        # 또는 raw_data['vr sickness response']가 0이 아닌 경우 (사용자가 스스로 멀미 수준을 보고한 시점)
        # 의 2가지 경우는 raw_data['score']를 정수화한다.
        # 그리고 해당 2가지 경우를 제외하면 보간된 값이므로 raw_data['score'] = np.nan으로 할당한다.
        # 그리고 np.nan 값들은 다음값을 이용하여 채워지도록 한다.
        max_time = raw_data['Elapsed'].max()
        target_times = np.arange(0, max_time + 1, 30)
        times = raw_data['Elapsed'].values
        idx = np.searchsorted(times, target_times)
        idx = np.clip(idx, 0, len(times) - 1)
        left_idx = np.clip(idx - 1, 0, len(times) - 1)
        dist_right = np.abs(times[idx] - target_times)
        dist_left = np.abs(times[left_idx] - target_times)
        final_indices = np.where(dist_left < dist_right, left_idx, idx)
        is_regular_report = np.zeros(len(raw_data), dtype=bool)
        is_regular_report[final_indices] = True

        is_spontaneous_report = raw_data['vr sickness response'] != 0
        is_valid_report = is_regular_report | is_spontaneous_report
        
        raw_data['score'] = raw_data['score'].where(is_valid_report, np.nan)
        raw_data['score'] = raw_data['score'].bfill()
        raw_data['score'] = raw_data['score'].ffill()
        
        filtered_data: pd.DataFrame = raw_data.iloc[:5000, :].copy()
        if "optical_flow" not in filtered_data.columns:
            filtered_data["optical_flow"] = np.nan
        filtered_data = filtered_data[["ID", "Lv", "frame", "prev_Elapsed", "Elapsed", "gaze-azimuth", "gaze-elevation",\
                                           'head-speed', 'head-ang_vel', 'score']]
        full_dataset.append(filtered_data)

full_dataset = pd.concat(full_dataset, ignore_index=True)
full_dataset.to_parquet(os.path.join(root_360_features, "total_filtered_data.parquet"))
display(full_dataset)

100%|██████████| 10/10 [00:00<00:00, 15.59it/s]


,ID,Lv,frame,prev_Elapsed,Elapsed,gaze-azimuth,gaze-elevation,head-speed,head-ang_vel,score
0,P01,Lv1,0,0.00,0.12,89.270157,19.633359,NaN,NaN,1.0
1,P01,Lv1,1,0.12,0.24,87.378063,19.876874,0.000000,0.032917,1.0
2,P01,Lv1,2,0.24,0.36,90.060824,19.572538,0.008333,0.020576,1.0
3,P01,Lv1,3,0.36,0.48,90.458815,2.120428,0.008333,0.019626,1.0
4,P01,Lv1,4,0.48,0.60,90.744803,0.401074,0.018634,0.201511,1.0
...,...,...,...,...,...,...,...,...,...,...
99983,P10,Lv2,4994,599.27,599.39,67.569929,-3.267630,0.008333,0.009542,3.0
99984,P10,Lv2,4995,599.39,599.51,70.423274,0.630266,0.000000,0.010146,3.0
99985,P10,Lv2,4996,599.51,599.63,70.060952,0.000000,0.000000,0.019442,3.0
99986,P10,Lv2,4997,599.63,599.75,69.826055,0.114592,0.000000,0.021400,3.0
